# Figyelemmechanizmusok és transzformerek

A rekurens hálózatok egyik fő hátránya, hogy a szekvencia minden szavának azonos hatása van az eredményre. Ez azzal jár, hogy a standard LSTM kódoló-dekódoló modellek nem teljesítenek optimálisan szekvencia-szekvencia feladatoknál, például a névelem-felismerésnél vagy a gépi fordításnál. Valójában az input szekvencia bizonyos szavai gyakran nagyobb hatással vannak a kimeneti szekvenciára, mint mások.

Vegyünk például egy szekvencia-szekvencia modellt, mint a gépi fordítás. Ez két rekurens hálózattal valósul meg, ahol az egyik hálózat (**kódoló**) az input szekvenciát egy rejtett állapotba sűríti, míg a másik, **dekódoló**, ezt a rejtett állapotot bontja ki a fordított eredményre. Ennek a megközelítésnek az a problémája, hogy a hálózat végső állapota nehezen tudja megjegyezni a mondat elejét, ami gyenge modellminőséget eredményez hosszú mondatok esetén.

**Figyelemmechanizmusok** lehetőséget adnak arra, hogy súlyozzuk az egyes input vektorok kontextuális hatását az RNN minden egyes kimeneti előrejelzésére. Ez úgy valósul meg, hogy rövidítéseket hozunk létre az input RNN köztes állapotai és a kimeneti RNN között. Ily módon, amikor a $y_t$ kimeneti szimbólumot generáljuk, figyelembe vesszük az összes input rejtett állapotot $h_i$, különböző súlykoefficiensekkel $\alpha_{t,i}$. 

![Kép egy kódoló/dekódoló modellről additív figyelemréteggel](../../../../../translated_images/hu/encoder-decoder-attention.7a726296894fb567.webp)
*A kódoló-dekódoló modell additív figyelemmechanizmussal [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), idézve [ebből a blogbejegyzésből](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

A figyelem mátrix $\{\alpha_{i,j}\}$ azt mutatja, hogy az egyes input szavak milyen mértékben játszanak szerepet egy adott szó generálásában a kimeneti szekvenciában. Az alábbiakban egy ilyen mátrix példáját láthatjuk:

![Kép egy mintaillesztésről, amelyet az RNNsearch-50 talált, Bahdanau - arviz.org](../../../../../translated_images/hu/bahdanau-fig3.09ba2d37f202a6af.webp)

*Ábra [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (3. ábra) alapján*

A figyelemmechanizmusok felelősek a természetes nyelvfeldolgozás jelenlegi vagy közel jelenlegi csúcsteljesítményéért. A figyelem hozzáadása azonban jelentősen növeli a modell paramétereinek számát, ami méretezési problémákat okozott az RNN-eknél. Az RNN-ek méretezésének egyik kulcsfontosságú korlátja, hogy a modellek rekurzív jellege megnehezíti a tanítás batch-elését és párhuzamosítását. Egy RNN-ben a szekvencia minden elemét sorrendben kell feldolgozni, ami azt jelenti, hogy nem lehet könnyen párhuzamosítani.

A figyelemmechanizmusok alkalmazása és ez a korlát vezettek a ma ismert és használt csúcsteljesítményű transzformer modellek létrehozásához, mint például a BERT és az OpenGPT3.

## Transzformer modellek

Ahelyett, hogy az előző előrejelzések kontextusát továbbítanák a következő értékelési lépésbe, a **transzformer modellek** **pozíciós kódolásokat** és figyelmet használnak, hogy megragadják az adott input kontextusát egy megadott szövegablakon belül. Az alábbi kép bemutatja, hogyan képesek a pozíciós kódolások és a figyelem megragadni a kontextust egy adott ablakon belül.

![Animált GIF, amely bemutatja, hogyan történik az értékelés a transzformer modellekben.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif) 

Mivel minden input pozíciót függetlenül térképeznek a kimeneti pozíciókhoz, a transzformerek jobban párhuzamosíthatók, mint az RNN-ek, ami lehetővé teszi sokkal nagyobb és kifejezőbb nyelvi modellek létrehozását. Minden figyelemfej különböző kapcsolatok megtanulására használható a szavak között, ami javítja a természetes nyelvfeldolgozási feladatok eredményét.

A **BERT** (Bidirectional Encoder Representations from Transformers) egy nagyon nagy, többrétegű transzformer hálózat, amelynek 12 rétege van a *BERT-base* esetében, és 24 a *BERT-large* esetében. A modellt először egy nagy szövegkorpuszra (WikiPedia + könyvek) tanítják be felügyelet nélküli tanítással (maszkolt szavak előrejelzése egy mondatban). Az előtanítás során a modell jelentős nyelvi megértést sajátít el, amelyet aztán más adathalmazokkal lehet finomhangolni. Ezt a folyamatot **transzfer tanulásnak** nevezzük. 

![Kép a http://jalammar.github.io/illustrated-bert/ oldalról](../../../../../translated_images/hu/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Számos transzformer architektúra létezik, például BERT, DistilBERT, BigBird, OpenGPT3 és még sok más, amelyek finomhangolhatók. A [HuggingFace csomag](https://github.com/huggingface/) lehetőséget biztosít ezeknek az architektúráknak a tanítására PyTorch segítségével. 

## BERT használata szövegklasszifikációhoz

Nézzük meg, hogyan használhatunk előre betanított BERT modellt hagyományos feladatunk megoldására: szekvencia klasszifikáció. Az eredeti AG News adathalmazunkat fogjuk osztályozni.

Először töltsük be a HuggingFace könyvtárat és az adathalmazunkat:


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


Mivel előre betanított BERT modellt fogunk használni, szükségünk lesz egy specifikus tokenizálóra. Először betöltünk egy tokenizálót, amely az előre betanított BERT modellhez tartozik.

A HuggingFace könyvtár tartalmaz egy előre betanított modellek gyűjteményét, amelyeket egyszerűen használhatsz, ha megadod a nevüket a `from_pretrained` függvények argumentumaként. Az összes szükséges bináris fájl a modellhez automatikusan letöltésre kerül.

Azonban bizonyos esetekben szükség lehet arra, hogy saját modelleket tölts be. Ilyenkor megadhatod annak a könyvtárnak az elérési útját, amely tartalmazza az összes releváns fájlt, beleértve a tokenizáló paramétereit, a `config.json` fájlt a modell paramétereivel, a bináris súlyokat stb.


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

A `tokenizer` objektum tartalmazza az `encode` függvényt, amely közvetlenül használható szöveg kódolására:


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Ezután hozzunk létre iterátorokat, amelyeket az edzés során az adatok eléréséhez fogunk használni. Mivel a BERT a saját kódolási függvényét használja, szükségünk lesz egy, a korábban definiált `padify`-hoz hasonló kitöltési függvény meghatározására:


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

Esetünkben egy előre betanított BERT modellt fogunk használni, amelyet `bert-base-uncased`-nek hívnak. Töltsük be a modellt a `BertForSequenceClassfication` csomag segítségével. Ez biztosítja, hogy a modellünk már rendelkezik a szükséges osztályozási architektúrával, beleértve a végső osztályozót is. Figyelmeztető üzenetet fogsz látni, amely szerint a végső osztályozó súlyai nincsenek inicializálva, és a modell előzetes betanítást igényel - ez teljesen rendben van, mivel pontosan ezt fogjuk most elvégezni!


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

Most már készen állunk az edzés megkezdésére! Mivel a BERT már előre betanított modell, érdemes viszonylag kicsi tanulási rátával kezdeni, hogy ne rontsuk el a kezdeti súlyokat.

A nehéz munka nagy részét a `BertForSequenceClassification` modell végzi. Amikor meghívjuk a modellt a tanító adatokon, az visszaadja az input minibatch-hez tartozó veszteséget és a hálózat kimenetét. A veszteséget a paraméterek optimalizálására használjuk (`loss.backward()` végzi a visszaterjesztést), míg a `out` segítségével számítjuk ki a tanítási pontosságot azáltal, hogy az `argmax`-szal kapott címkéket (`labs`) összehasonlítjuk a várt `labels` értékekkel.

A folyamat ellenőrzése érdekében a veszteséget és a pontosságot több iteráción keresztül halmozzuk, majd minden `report_freq` tanítási ciklus után kiírjuk azokat.

Ez a tanítás valószínűleg elég hosszú időt vesz igénybe, ezért korlátozzuk az iterációk számát.


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


Látható (különösen, ha növeled az iterációk számát és elég sokáig vársz), hogy a BERT osztályozás elég jó pontosságot ad! Ez azért van, mert a BERT már eleve elég jól érti a nyelv szerkezetét, és nekünk csak a végső osztályozót kell finomhangolnunk. Azonban, mivel a BERT egy nagy modell, az egész tanítási folyamat sok időt vesz igénybe, és komoly számítási kapacitást igényel! (GPU, és lehetőleg több is).

> **Megjegyzés:** Példánkban az egyik legkisebb előre betanított BERT modellt használtuk. Vannak nagyobb modellek is, amelyek valószínűleg jobb eredményeket hoznak.


## A modell teljesítményének értékelése

Most kiértékelhetjük a modellünk teljesítményét a tesztadatokon. Az értékelési ciklus nagyon hasonló a tanítási ciklushoz, de ne felejtsük el, hogy a modellt értékelési módba kell kapcsolni a `model.eval()` hívásával.


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## Legfontosabb tanulság

Ebben az egységben láthattuk, milyen egyszerű a **transformers** könyvtárból származó előre betanított nyelvi modellt adaptálni a szövegklasszifikációs feladatunkhoz. Hasonlóképpen, a BERT modellek használhatók entitáskinyerésre, kérdés-megválaszolásra és más NLP-feladatokra.

A transzformer modellek jelenleg az NLP csúcstechnológiáját képviselik, és a legtöbb esetben ezekkel érdemes először kísérletezni, amikor egyedi NLP-megoldásokat valósítunk meg. Azonban rendkívül fontos megérteni az ebben a modulban tárgyalt rekurzív neurális hálózatok alapelveit, ha fejlettebb neurális modelleket szeretnénk építeni.



---

**Felelősségkizárás**:  
Ezt a dokumentumot az [Co-op Translator](https://github.com/Azure/co-op-translator) AI fordítószolgáltatás segítségével fordítottuk le. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Kritikus információk esetén javasolt a professzionális, emberi fordítás igénybevétele. Nem vállalunk felelősséget a fordítás használatából eredő félreértésekért vagy téves értelmezésekért.
